In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time
import copy


In [2]:
def L_model_forward(X, parameters):

    x0 = X
    caches = {}
#    caches = []
    # Gotta save the inputs as the first activations
    caches['A0'] = x0
#    print('Input size..'+str(x0.shape))

    for i in range(1, int(len(parameters.keys())/2)+1):
#        print('Max value in W'+str(i)+'....'+str(np.max(parameters['W'+str(i)].flatten())))
        zl = np.matmul(parameters['W'+str(i)], x0) + parameters['b'+str(i)]
        if (i == int(len(parameters.keys())/2)):
            # Output layer Activation is sigmoid        
            x0 = ex_activations.sigmoid(zl)
#            print('Fwd prop Layer..'+str(i)+'...Sigmoid..'+str(parameters['W'+str(i)].shape))            
        else:            
#            print('Fwd prop Layer..'+str(i)+'...Relu..'+str(parameters['W'+str(i)].shape))            
            x0 = ex_activations.relu(zl)

        caches['A'+str(i)]=x0
        caches['Z'+str(i)]=zl
        caches['W'+str(i)]=parameters['W'+str(i)]
        caches['b'+str(i)]=parameters['b'+str(i)]        

    x0[x0 <= 0.001] = 0.001
    x0[x0 >= 0.999] = 0.999

    AL = x0     
    
#    print('Caches..', len(caches))
    return AL, caches 


In [3]:
def initialize_parameters_deep(layers_dims):

    num_layers = len(layers_dims)
#    print(num_layers)
    parameters = {}
    
    for i in range(num_layers-1):
        parameters['W'+str(i+1)] = np.random.randn(layers_dims[i+1], layers_dims[i]) * np.sqrt(2/(layers_dims[i+1]+layers_dims[i]))
        parameters['b'+str(i+1)] = np.zeros((layers_dims[i+1], 1))
#        print(parameters['b'+str(i+1)].shape)
        
    return parameters 



In [4]:
def update_parameters(parameters, grads, learning_rate):

    for i in range(1, int(len(parameters.keys())/2)+1):
#        print('Updating Layer..'+str(i))
        parameters['W'+str(i)] = parameters['W'+str(i)] - (learning_rate * grads['dW'+str(i)])
        parameters['b'+str(i)] = parameters['b'+str(i)] - (learning_rate * grads['db'+str(i)])

    return parameters



In [5]:
# Backward propagation (Sigmoid -> Relu -> Relu..)
def L_model_backward(AL, Y, caches):

    grads = {}
    m = len(Y.flatten())
    # Condition the shapes so that output and labels align
    Y = np.reshape(Y, (1,m))
    AL = np.reshape(AL, (1,m))
    numlayers = int(np.floor(len(caches)/4))   

    da_next = np.divide(1-Y, 1-AL) - (np.divide(Y, AL))

    for i in range(numlayers, 0, -1):
        # Last layer has a different activation
        if (i == numlayers):
            gprime_curr = ex_activations.sigmoidDeriv(caches['Z'+str(i)])
        else:
            gprime_curr = ex_activations.reluDerivative(caches['Z'+str(i)])

        dz = np.multiply(da_next, gprime_curr)

        # Activations coming into the first layer are the inputs
        temp_al_prev = np.transpose(caches['A'+str(i-1)])

        dW = np.divide(np.matmul(dz, temp_al_prev), m)
        db = np.divide(np.sum(dz, axis=1, keepdims=True), m)
        
        da_prev = np.matmul(np.transpose(caches['W'+str(i)]), dz)
        
        da_next = da_prev
        grads['dW'+str(i)] = dW
        grads['db'+str(i)] = db

    return grads


In [6]:
# Backward propagation (Sigmoid -> Relu -> Relu..)
def L_model_backward_L2_reg(AL, Y, caches, L2_lambd=200):

    grads = {}
    m = len(Y.flatten())
    # Condition the shapes so that output and labels align
    Y = np.reshape(Y, (1,m))
    AL = np.reshape(AL, (1,m))
    numlayers = int(np.floor(len(caches)/4))   

    da_next = np.divide(1-Y, 1-AL) - (np.divide(Y, AL))

    for i in range(numlayers, 0, -1):
        # Last layer has a different activation
        if (i == numlayers):
            gprime_curr = ex_activations.sigmoidDeriv(caches['Z'+str(i)])
        else:
            gprime_curr = ex_activations.reluDerivative(caches['Z'+str(i)])

        dz = np.multiply(da_next, gprime_curr)

        # Activations coming into the first layer are the inputs
        temp_al_prev = np.transpose(caches['A'+str(i-1)])

        dW = np.divide(np.matmul(dz, temp_al_prev), m)
        # Gradient for W here is L2 regularized
        L2term = (L2_lambd * np.sum(caches['W'+str(i)]))/m
        dW += L2term
        
        db = np.divide(np.sum(dz, axis=1, keepdims=True), m)
        
        da_prev = np.matmul(np.transpose(caches['W'+str(i)]), dz)
        
        da_next = da_prev
        grads['dW'+str(i)] = dW
        grads['db'+str(i)] = db

    return grads


In [7]:
# Logistic regression cost for now
def compute_cost(AL, Y):
    
#    cost = np.sum(np.multiply((1-Y),np.log((1-AL))) + np.multiply((Y),np.log((AL))))
    m = len(Y.flatten())
    # Condition the shapes so that output and labels align
    Y = np.reshape(Y, (1,m))
    AL = np.reshape(AL, (1,m))
    
    cost = -1/m * (np.sum(np.multiply(1-Y, np.log(1-AL))) + np.sum(np.multiply(Y, np.log(AL))))
    
    # Also return success rate
    x = (AL >= 0.5)

    success = 1 - (np.count_nonzero(Y - x)/m)
    
    return cost, success


In [8]:
# Logistic regression cost for now
def compute_L2_reg_cost(AL, Y, parameters, L2_lambd = 200):
    
#    cost = np.sum(np.multiply((1-Y),np.log((1-AL))) + np.multiply((Y),np.log((AL))))
    m = len(Y.flatten())
    # Condition the shapes so that output and labels align
    Y = np.reshape(Y, (1,m))
    AL = np.reshape(AL, (1,m))

    # Cross entropy cost    
    cost = -1/m * (np.sum(np.multiply(1-Y, np.log(1-AL))) + np.sum(np.multiply(Y, np.log(AL))))
    
    # Compute the L2 reg cost component
    L2cost = 0
    for i in range(1, int(len(parameters.keys())/2)+1):
        L2cost += np.sum(np.square(parameters['W'+str(i)]))

    L2cost = (L2_lambd * L2cost)/(2*m)
    
    # Also return success rate
    x = (AL >= 0.5)

    success = 1 - (np.count_nonzero(Y - x)/m)
    
    return (cost + L2cost), success


In [9]:
## Global variables are defined in the class definition
class Deep_NN_frmwrk:
    
    def __init__(self, name):
        self.name = name
        self.localfolder = '/Users/nkolli/Documents/Python/Cats_vs_Dogs'
        # Define input feature length - number of pixels in the images here
        self.img_h = 32
        self.img_w = 32
        self.img_d = 1
    
        dim_1 = self.img_h * self.img_w * self.img_d
        self.layers_dims = [dim_1, 25, 13, 5, 1] #  5-layer model
    
        ### Train data
        self.kfolds = 10
        self.numbatches = 20
        self.learning_rate = 0.001
        self.num_epochs = 300
        self.print_cost = 1
        self.use_saved_params = 1
        self.L2_lambd = 0.7

    def loadSavedParams(self):
    
        num_layers = len(self.layers_dims)
        
        ### Load parameters from training
        parameters = {}
        for i in range(num_layers-1):
            parameters['W'+str(i+1)] = np.loadtxt(self.localfolder+'/Out_W'+str(i+1))
            if (len(parameters['W'+str(i+1)].shape) < 2):
                parameters['W'+str(i+1)] = np.reshape(parameters['W'+str(i+1)], (1, len(parameters['W'+str(i+1)])))
                
            parameters['b'+str(i+1)] = np.loadtxt(self.localfolder+'/Out_b'+str(i+1))
            if (len(parameters['b'+str(i+1)].shape) < 1):
                parameters['b'+str(i+1)] = np.reshape(parameters['b'+str(i+1)], (1, 1))
            else:
                parameters['b'+str(i+1)] = np.reshape(parameters['b'+str(i+1)], (len(parameters['b'+str(i+1)]), 1))            
        
        return parameters
    
    def saveParams(self, params):
        # Save parameters from training
        for i in range(1,int(len(params.keys())/2)+1):
            np.savetxt(self.localfolder+'/Out_W'+str(i), params['W'+str(i)])
            np.savetxt(self.localfolder+'/Out_b'+str(i), params['b'+str(i)])
                
            
    def showImageData(X, Y, t_size):
        
    #    # To view the images, make sure labeling is correct
        imgi = int(input('Enter an index..'))
        while (imgi != 99):
            print('Index chosen is...', imgi)
            print('Label is...', Y[imgi])
            reimg = Image.fromarray(np.reshape(X[:,imgi],(t_size)))
    #        t_size = (320,320)
    #        reimg = reimg.resize(t_size, Image.ANTIALIAS)
            plt.imshow(reimg)
            imgi = int(input('Enter an index..'))
    
    def splitKfold(self, X, Y, j, m):
    
        # K-fold xval, separate into train and x_val
        # If no kfold just use 10% as xval
        if (self.kfolds == 1):
            xval = int(m/10)
            x_train = X[:,0:m-xval]
            y_train = Y[0:m-xval]    
            x_xval = X[:,m-xval+1:]
            y_xval = Y[m-xval+1:]
        else:
            xval = int(m/self.kfolds)
            tmp_range = range(j*xval, (j+1)*xval)
            x_xval = X[:, tmp_range]
            y_xval = Y[tmp_range]
            # splits X and Y
            if (j==0):
                tmp_range = range((j+1)*xval+1, m)
                x_train = X[:, tmp_range]
                y_train = Y[tmp_range]
            else:
                tmp_range1 = range(0, j*xval)
                tmp_range2 = range((j+1)*xval+1, m)
                x_train = np.concatenate((X[:, tmp_range1], X[:, tmp_range2]), axis=1)
                y_train = np.concatenate((Y[tmp_range1], Y[tmp_range2]), axis=0)
                
    #    print(x_xval.shape, y_xval.shape, x_train.shape, y_train.shape)
        return x_xval, y_xval, x_train, y_train
    
    def L_layer_model(self, X, Y, parameters):#lr was 0.009
        """
        Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
        
        Arguments:
        X -- data, numpy array of shape (number of examples, num_px * num_px * 3)
        Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
        layers_dims -- list containing the input size and each layer size, of length (number of layers + 1).
        learning_rate -- learning rate of the gradient descent update rule
        num_iterations -- number of iterations of the optimization loop
        print_cost -- if True, it prints the cost every 100 steps
        
        Returns:
        parameters -- parameters learnt by the model. They can then be used to predict.
        """
    
        # NK: keep this for now, for testing the model
    #    np.random.seed(1)
        costs = []                         # keep track of cost
        train_successes = []
        xval_successes = []
        # Parameters initialization
        if (len(parameters) == 0):
            parameters = ex_init_layer_weights.initialize_parameters_deep(self.layers_dims)
    
        newparams = copy.deepcopy(parameters)
        
        # Separate ino Training and Cross validation sets
        m = X.shape[1]
            
        starttime = time.time()
        # Loop (gradient descent)
        for i in range(0, int(self.num_epochs/self.kfolds)): 
    
            for j in range(0, self.kfolds):
                
                X_xval, Y_xval, X_train, Y_train = self.splitKfold(X, Y, j, m)
                # Batch training, numbatches is 1 if no batch separation
                batch_size = int(X_train.shape[1]/self.numbatches)
            
                for k in range(0, self.numbatches):
                    
                    X_batch = X_train[:,(k*batch_size):((k+1)*batch_size-1)]
                    Y_batch = Y_train[(k*batch_size):((k+1)*batch_size-1)]
        
                    # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.            
                    AL, caches = ex_fwd_prop.L_model_forward(X_batch, newparams)
                    
                    # Compute cost
#                    cost, train_success = ex_compute_cost.compute_cost(AL, Y_batch)
                    # Compute cost with L2 reg
                    cost, train_success = ex_compute_cost.compute_L2_reg_cost(AL, Y_batch, newparams, self.L2_lambd)
                
                    # Backward propagation
#                    grads = ex_back_prop.L_model_backward(AL, Y_batch, caches)
                    # Backward propagation with L2 reg
                    grads = ex_back_prop.L_model_backward_L2_reg(AL, Y_batch, caches, self.L2_lambd)
             
                    # Update parameters.
                    newparams = ex_update_parameters.update_parameters(newparams, grads, self.learning_rate)
                    
            # Print the cost every nth training cycle
            if self.print_cost and i % 1 == 0:
                endtime = time.time()
                print ("Cost after iteration %i: %f, Time: %f" %(i, cost, (endtime-starttime)))
                costs.append(cost)
                train_successes.append(train_success)
                # Run xval set and compute cost.
                AL_xval, xval = ex_fwd_prop.L_model_forward(X_xval, newparams)        
                xval_cost, xval_success = ex_compute_cost.compute_cost(AL_xval, Y_xval)
                xval_successes.append(xval_success)
                # plot the cost
                plt.plot(np.squeeze(train_successes), 'b')
                plt.plot(np.squeeze(xval_successes), 'k')            
                plt.ylabel('Success %')
                plt.xlabel('iterations (per tens)')
                plt.title("Learning rate =" + str(self.learning_rate))
                plt.draw()
                plt.pause(1)
                # Optional save parameters every nth iteration for future use
                self.saveParams(newparams)
    
                
        return parameters, newparams, train_success
    
    def train_data(self):
        
        # Load dataset 
        X_train = np.loadtxt(self.localfolder+'/X_train.dat')
        Y_train = np.loadtxt(self.localfolder+'/Y_train.dat')
        
        # Number of examples
        m = X_train.shape[1]
        print('Data loaded..' + str(time.ctime()) + '...' + str(X_train.shape))
        
        # Shuffle the data
        marr = np.arange(m-1)
        np.random.shuffle(marr)
        X_train = X_train[:,marr]
        Y_train = Y_train[marr]
        
        X_train = X_train/255
    #    X_train[X_train <= 0.4] = 0
    #    X_train[X_train > 0.4] = 1
        
        print('Layer dims...'+str(self.layers_dims) + '...' + str(time.ctime()))
        parameters = {}
        if (self.use_saved_params):
            parameters = self.loadSavedParams()
    
        params, newparams, train_success = self.L_layer_model(X_train, Y_train, parameters)
        print('Optimization done..'+str(train_success))
        
        self.saveParams(newparams)
    
            
    def test_data(self, t_size):
        
        # Load test data
        X_test = np.loadtxt(self.localfolder+'/X_test.dat')
        Y_test = np.loadtxt(self.localfolder+'/Y_test.dat')
        
        # Number of examples
        m = X_test.shape[1]
        
        # Shuffle the data
        marr = np.arange(m-1)
        np.random.shuffle(marr)
        X_test = X_test[:,marr]
        Y_test = Y_test[marr]
        
        X_test = X_test/255
    #    X_train[X_train <= 0.4] = 0
    #    X_train[X_train > 0.4] = 1
        
        parameters = self.loadSavedParams()
        
        ### Run forward prop to get the output activations    
        AL, caches = ex_fwd_prop.L_model_forward(X_test, parameters)
        
        ### Convert AL to binary calls
        calls = (AL >= 0.5)
        
        ## Calculate success percentage
        success = 1 - np.count_nonzero(Y_test - calls)/m
    
        # Show images from test data and the classification results    
#        self.showImageData(X_test, calls, (t_size))    
        
        return success
